### Imports

In [8]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import linalg as LA
from ipywidgets import *
from manim import *

### Crossection color map animation


In [9]:
%%manim -ql -v WARNING --disable_caching ColorMapCrossSection
#creates animation of crosssections fadind into each other


class ColorMapCrossSection(Scene):
    def construct(self):
        
        #data array goes in here shape= (x,y,z) where z is the time axis
        # Generate 3D array data with random integers between 1 and 5
        x_size, y_size, z_size = 300, 300, 50
        data = np.random.randint(1, 3, (x_size, y_size, z_size))
        
        #creates the images (can be done outside of scene if correct naming is used)
        # Function to generate a color map from 2D data and save it as an image
        def create_colormap_image(data_slice, z_index):
            plt.imshow(data_slice, cmap='viridis', vmin=1, vmax=5)
            plt.axis('off')  # Remove axis
            # Save the plot as an image
            plt.savefig(f"junk/slice_{z_index}.png", bbox_inches='tight', pad_inches=0)
            plt.close()

        # Create color map images for each slice along the z-axis
        for z_slice in range(z_size):
            create_colormap_image(data[:, :, z_slice], z_slice)

        ## mAnimation
        # Load the first image to display it
        #starting image
        run_time = 0.2 #speed of transitions
        #wait_time = 2 #time between transitions (not needed)
        image_mobject = ImageMobject("junk/slice_0.png").scale(3)  # Scale to desired size
        self.play(FadeIn(image_mobject),run_time=run_time)  # Show the first slice
        # self.wait(wait_time)

        # Transition through the z slices
        for z_slice in range(1, z_size):
            new_image_mobject = ImageMobject(f"junk/slice_{z_slice}.png").scale(3)
            self.play(Transform(image_mobject, new_image_mobject),run_time=run_time) 
            #self.wait(wait_time)  # Pause to observe each slice

        self.wait(2) #display end for a little lnger


Manim Community v0.18.1

KeyboardInterrupt: 

### Orbit plot

#### Creates example orbit

In [10]:
#import needed: numpy, pyplot, plotLayout
def nd_orbit(fnct, x0, it):
    # calculates orbits of n dimensional map
    # Where:
    #- param fnct: Mapping function mapping x0 to x1 taking a single parameter
    #- param x0: Initial value
    #- param it: Number of iterations
    #- returns: The entire orbit, including the initial value
    #   Initialize
    orbit = np.zeros((len(x0),it))
    orbit[:,0]=x0
    #   Main loop
    for i in range(it-1):
        orbit[:,i+1] = fnct(orbit[:,i])
    return orbit

In [11]:
def activation(x):
    #the actication  function f()
    #return (x+abs(x))/2 #ReLu
    return 1/(1+np.exp(-x)) #sigmoid 
    #return np.tanh(x) #tanh

In [12]:
#import needed: numpy, pyplot, plotLayout, nd_orbit
def neuron_2_discrete(x0,a,b,alpha ,Nit, ntrans):
    # calculates stable solution henon map
    # Where:
    #- param x0: Initial value
    #- param a: Parameters of Henon map, being [a,b] in the standard expression
    #- returns: An array containing the orbit of the Henon map without transients

    #   Initialize  
    linearmap = lambda xy: alpha * (a @ activation(xy) + b)
    n_iter = Nit
    # n_steps_until_stable = 100

    #   Calculate orbit
    orbit = nd_orbit(linearmap, x0, n_iter)
    orbit = orbit[:, ntrans+1:] #remove the transient (+1 to also remove initial condition)
    return orbit


In [14]:
def orbit_generate():
    x0 = -np.array([0.1,0.1])
    w11 = -10
    w12 = 5.9
    a = np.array([[w11,w12],
                [-6.6,0]])
    b = np.array([-3.8,3])
    alpha = 1
    orbit = neuron_2_discrete(x0,a,b,alpha,20,0)
    return orbit

#### Manimation

In [19]:
%%manim -ql -v WARNING --disable_caching plorbit

class plorbit(Scene):
    def calculate_axis_ranges(self, points):
        """
        Calculate the x and y axis ranges based on the min and max
        values of the orbit points.
        """
        # Extract x and y coordinates
        x_values = [h[0] for h in points]
        y_values = [h[1] for h in points]

        # Calculate min and max for x and y
        x_min, x_max = min(x_values), max(x_values)
        y_min, y_max = min(y_values), max(y_values)

        # Add padding to the axis ranges for better visualization
        padding_factor = 0.1  # 10% padding on each side
        x_padding = (x_max - x_min) * padding_factor
        y_padding = (y_max - y_min) * padding_factor

        # Return the padded ranges
        return x_min - x_padding, x_max + x_padding, y_min - y_padding, y_max + y_padding
    def animate(self,orbit):
        # Create axes for the orbit plot
        x_min, x_max, y_min, y_max = self.calculate_axis_ranges(orbit)

        # Create axes dynamically based on the calculated ranges
        axes = Axes(
            x_range=[x_min, x_max, (x_max - x_min) / 4],  # Divide into 4 intervals
            y_range=[y_min, y_max, (y_max - y_min) / 4],
            x_length=6,  # Length in units
            y_length=6,
            axis_config={"color": BLUE}
        )
        # axes = Axes(
        #     x_range=[-2, 2, 0.5], y_range=[-2, 2, 0.5],
        #     x_length=6, y_length=6,
        #     axis_config={"color": BLUE}
        # )
        labels = axes.get_axis_labels(x_label="h1", y_label="h2")
        numberplane = NumberPlane()
        self.add(axes, labels)
        #self.play(Create(axes), Write(labels))
        # Convert the simulated points to Manim points on the 2D plane
        orbit_points_manim = [
            axes.c2p(orbit[0,i],orbit[1,i]) for i in range(orbit.shape[1])
        ]
        # Create the orbit as a line with a moving dot
        orbit_path = VMobject(color=BLUE)
        orbit_path.set_points_as_corners(orbit_points_manim)
        moving_dot = Dot(orbit_points_manim[0], color=RED)

        # Animation: Draw the orbit path and move the dot along the path
        self.play(Create(orbit_path), FadeIn(moving_dot))
        self.play(MoveAlongPath(moving_dot, orbit_path), run_time=4, rate_func=linear)

        # Hold the final state for a moment
        self.wait(2)
    def construct(self):
        orbit_points = orbit_generate()
        self.animate(orbit_points)


Manim Community v0.18.1